In [1]:
import pandas as pd
import dask.dataframe as dd
import os
import pandas as pd
from glob import glob
from tqdm import tqdm
import tarfile
import shutil

# 1. sqlite3 

In [3]:

# import sqlite3 as sql

# csvs_dir = '/Users/personal-macbook/Documents/projects/inaturalist/species_csv'
# csv_list = glob(f'{csvs_dir}/*.csv')

# df = pd.DataFrame(csv_list, columns=['csv_list'])
# df['downloaded'] = False
# df.reset_index(inplace=True)

# df.to_csv(f'{csvs_dir}/csv_list.csv', index=False)

# saving the csv as sql database
# df.to_sql('csv_list_table', con=sql.connect(f'{csvs_dir}/sql_inaturalist_insects.db'), if_exists='replace')

#  convert df to sql database
# df = pd.read_csv(f'csv_list.csv', index_col=['index'])

# df.to_sql('csv_list_table', con=sql.connect(f'{csvs_dir}/sql_inaturalist_insects.db'), if_exists='replace')

# modify a cell in column downlaoded of table csv_list_table in sql_inaturalist_insects.db
# index = 4

# sqliteConnection = sql.connect(f'{csvs_dir}/sql_inaturalist_insects.db')
# cursor = sqliteConnection.cursor()
# sqlite_inser_query = f"UPDATE csv_list_table SET downloaded = 1 WHERE index = {index}"


# # cursor.execute(f'UPDATE csv_list_table SET downloaded = 1 WHERE index = {index}')


# cursor.execute(sqlite_inser_query)
# sqliteConnection.commit()
# sqliteConnection.close()

# pd.read_sql_query('SELECT * FROM csv_list_table', con=sql.connect(f'{csvs_dir}/sql_inaturalist_insects.db'))

# 2. Renaming the taxon_id to their names

## 2.1 Extracting the taxon_ids for insecta images

### 2.1.1 read taxa.csv file

In [61]:
dir = '/Users/personal-macbook/Documents/projects/inaturalist/data/full_data_csv_lists'

# df_taxa = dd.read_csv(f'{dir}/taxa.csv', sep='	', dtype=str)
df_taxa = pd.read_csv(f'{dir}/taxa.csv', sep='	', dtype=str)
df_taxa

,taxon_id,ancestry,rank_level,rank,name,active
0,8943,48460/1/2/355675/3/7251/980017,20,genus,Calyptura,true
1,8605,48460/1/2/355675/3/7251/71348,20,genus,Lophorina,true
2,8856,48460/1/2/355675/3/7251/8659/8855,10,species,Porphyrolaema porphyrolaema,true
3,27,48460/1/2/355675/3/4/23/24,10,species,Grus canadensis,false
4,8784,48460/1/2/355675/3/7251/8659/8756,10,species,Pipreola whitelyi,true
...,...,...,...,...,...,...
1320219,1361890,48460/47126/211194/47125/47124/47363/47362/373...,10,species,Suberogerens cyclophylla,true
1320220,1361889,48460/47126/211194/47125/47124/47363/47362/373...,20,genus,Suberogerens,true
1320221,1361896,48460/48222/348288/348289/1359686/348290/96081...,10,species,Uvigerina peregrina,true
1320222,1361897,48460/47126/211194/47125/47163/47218/47217/738...,10,species,Trichocentrum natalieae,true


### 2.1.2 Extracting the ancestors taxon_id map for insecta
```
48460/1/47120/372739/47158
```

In [71]:
ancestries = '48460/1/47120/372739/47158'.split('/')
df_taxa_insecta_ancestors = df_taxa.loc[df_taxa.taxon_id.isin( ancestries)]
df_taxa_insecta_ancestors

,taxon_id,ancestry,rank_level,rank,name,active
26350,47158,48460/1/47120/372739,50,class,Insecta,true


### 2.1.3 Extracting the images under insecta

In [12]:
df_taxa[df_taxa.isna()] = '-'

df_taxa_insecta = df_taxa.loc[df_taxa.ancestry.str.contains('48460/1/47120/372739/47158')]

df_taxa_insecta

,taxon_id,ancestry,rank_level,rank,name,active
18978,49089,48460/1/47120/372739/47158/184884/47208/71130/...,10,species,Kheper aeratus,true
18985,49196,48460/1/47120/372739/47158/184884/47157/176535...,10,species,Ectodemia castaneae,false
19038,47860,48460/1/47120/372739/47158/184884/47864/47863/...,10,species,Archichauliodes conversus,true
19039,49993,48460/1/47120/372739/47158/184884/47157/47224/...,5,subspecies,Heliconius charitonius churchi,false
19046,49988,48460/1/47120/372739/47158/184884/47157/47224/...,5,subspecies,Heliconius charitonius bassleri,false
...,...,...,...,...,...,...
1320212,1361879,48460/1/47120/372739/47158/184884/47201/124417...,10,species,Trigonopsis neotropica,true
1320213,1361882,48460/1/47120/372739/47158/184884/47201/124417...,10,species,Trigonopsis schunkei,true
1320214,1361891,48460/1/47120/372739/47158/184884/47157/47607/...,10,species,Cosmosoma lignicolor,true
1320216,1361898,48460/1/47120/372739/47158/184884/47822/48091/...,10,species,Procecidochares lisae,true


### 2.1.4 Merging the two tables

In [14]:
df = pd.concat( (df_taxa_insecta_ancestors, df_taxa_insecta), axis=0)
df.to_csv('data/taxa_insecta.csv', index=True)
df

,taxon_id,ancestry,rank_level,rank,name,active
4953,1,48460,70,kingdom,Animalia,true
26350,47158,48460/1/47120/372739,50,class,Insecta,true
26986,48460,NaN,100,stateofmatter,Life,true
27182,47120,48460/1,60,phylum,Arthropoda,true
338187,372739,48460/1/47120,57,subphylum,Hexapoda,true
...,...,...,...,...,...,...
1320212,1361879,48460/1/47120/372739/47158/184884/47201/124417...,10,species,Trigonopsis neotropica,true
1320213,1361882,48460/1/47120/372739/47158/184884/47201/124417...,10,species,Trigonopsis schunkei,true
1320214,1361891,48460/1/47120/372739/47158/184884/47157/47607/...,10,species,Cosmosoma lignicolor,true
1320216,1361898,48460/1/47120/372739/47158/184884/47822/48091/...,10,species,Procecidochares lisae,true


## 2.2 Renaming the folders corresponding to ancestories

In [55]:
def rename_taxon_id_to_name(dir, df, level):
    """ renmaing taxon_id to taxon_name for all folders (not images) """
    
    # getting all subfolders inside the directory
    dir, subfolders, subfiles = next(os.walk(dir))
    print('level: ', level)
    
    for subfl in subfolders:
        
        # finding the mapping for taxon id to name
        taxon_name = df.loc[df.taxon_id == subfl, 'name'].values[0]
        
        # renmaing the folder
        os.rename(f'{dir}/{subfl}', f'{dir}/{taxon_name}')
        
        # contiuing the process for subfolders
        rename_taxon_id_to_name(f'{dir}/{taxon_name}', df, level+1)
        
        
dir_data_main = '/Users/personal-macbook/Documents/projects/inaturalist/data/uncompressed'

rename_taxon_id_to_name(dir_data_main, df, 0)


In [83]:
os.path.isdir('code/data')

False

# Reviewing the csv metadata

### Observations
Note: each observer_id can be repeated multiple times in this dataframe

In [5]:
df_observations = dd.read_csv(f'{dir}/observations.csv', sep='	', dtype=str)
df_observations_filt = df_observations[['taxon_id', 'quality_grade']]

df_observations.head()

df_observations[df_observations.observation_uuid == '526ababd-aff2-4ce8-b111-d8013b9d8b42'].head()

/Users/personal-macbook/opt/anaconda3/envs/aws/lib/python3.8/site-packages/dask/dataframe/core.py:7296: UserWarning: Insufficient elements for `head`. 5 elements requested, only 1 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


,observation_uuid,observer_id,latitude,longitude,positional_accuracy,taxon_id,quality_grade,observed_on
0,526ababd-aff2-4ce8-b111-d8013b9d8b42,505,38.6097400000,-122.7673020000,NaN,53137,research,2011-05-30


### observers

In [6]:
df_observers = dd.read_csv(f'{dir}/observers.csv', sep='	', dtype=str)
df_observers.head()

,observer_id,login,name
0,1,kueda,Ken-ichi Ueda
1,8,tueda,NaN
2,28,tiwane,Tony Iwane
3,85,russiannaturalistbrazil,NaN
4,129,field_daze,Liv MW


### photos

In [7]:
df_photos = dd.read_csv(f'{dir}/photos.csv', sep='	', dtype=str)
df_photos_filt = df_photos[['photo_id', 'license','extension','photo_uuid']]
df_photos.head()

# df_photos.set_index('observation_uuid').head()
df_photos[df_photos.observation_uuid == '526ababd-aff2-4ce8-b111-d8013b9d8b42'].head()

/Users/personal-macbook/opt/anaconda3/envs/aws/lib/python3.8/site-packages/dask/dataframe/core.py:7296: UserWarning: Insufficient elements for `head`. 5 elements requested, only 3 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


,photo_uuid,photo_id,observation_uuid,observer_id,extension,license,width,height,position
0,9b5ca6f4-fb23-47ff-b581-5fe3f9d068aa,35927,526ababd-aff2-4ce8-b111-d8013b9d8b42,505,jpg,CC-BY,2048,1356,0
1,a291d420-3ca6-4f94-b955-6c936a9b596d,35926,526ababd-aff2-4ce8-b111-d8013b9d8b42,505,jpg,CC-BY,2048,1680,1
2,c6d7769c-0390-42cc-8012-874cd49ea180,35925,526ababd-aff2-4ce8-b111-d8013b9d8b42,505,jpg,CC-BY,2048,1863,2


In [8]:
# df_observations[df_observations.observation_uuid == '526ababd-aff2-4ce8-b111-d8013b9d8b42'].head()
df22 = dd.merge(left=df_photos, right=df_observations, how='left', on='observation_uuid')
# df22 = df_observations.drop(columns=['observer_id'])
# df2 = dd.concat([df_photos, df22])
# df2.head()

In [71]:
# import sklearn 
# from sklearn.datasets import load_iris
# data = load_iris()
# df = pd.DataFrame(data['data'] , columns=data['feature_names'])

# df1 = df[df.columns[:3]]
# df2 = df[df.columns[2:]]
# pd.concat([df1,df2],axis=1)


In [38]:
# from dask import delayed

# def test(x): 
#     return x**3 + 1

# test_delayed = delayed(test)(3)

# d = 4*test_delayed - 3
# print(d.compute())


109


# Renaming the images to their respective photo-id

shards_list = [ l for l in os.listdir(ancestry) if os.path.isdir(ancestry + '/' + l)]

import json
from glob import glob

for shard in shards_list:
    
    json_list = [l.split('/')[-1].split('.json')[0] for l in glob(f'{ancestry}/{shard}/*.json')]
    
    dir_img = f'{ancestry}/{shard}'
    
    for img_new_id in json_list:

        f = open(f'{dir_img}/{img_new_id}.json', 'r')
        url = json.load(f)['url']
        
        photo_id = url.split('/')[-2]
        img_size_n_format_orig = url.split('/')[-1]
        
        img_size = img_size_n_format_orig.split('.')[0]
        img_format = 'jpg' if CONVERT_IMAGE_FORMAT else img_size_n_format_orig.split('.')[1]

        os.rename(f'{dir_img}/{img_new_id}.{img_format}', f'{dir_img}/{photo_id}_{img_size_n_format_orig}')
        os.rename(f'{dir_img}/{img_new_id}.json', f'{dir_img}/{photo_id}_{img_size}.json')
            
